# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from config import weather_api_key, gkey
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

618

In [3]:
cities_list = []
for city in cities: 
    cities_list.append(city.title())
    
print(cities_list)

['Coquimbo', 'Port Elizabeth', 'Hithadhoo', 'Veraval', 'Beloha', 'Atar', 'Khatanga', 'Airai', 'Illoqqortoormiut', 'Mys Shmidta', 'Butaritari', 'Busselton', 'Carnarvon', 'Punta Arenas', 'Bethel', 'Plettenberg Bay', 'Fortuna', 'Atyashevo', 'Maputo', 'Kapaa', 'Gunjur', 'Bara', 'Qena', 'Chokurdakh', 'Hermanus', 'Pevek', 'Alsfeld', 'Peleduy', 'Namatanai', 'Zelenoborsk', 'Alyangula', 'Olds', 'Bilibino', 'Albany', 'Georgetown', 'Kiunga', 'Aleksandrovka', 'Barrow', 'Sao Filipe', 'Snezhnogorsk', 'Avarua', 'Kaitangata', 'Malanje', 'Mehamn', 'Codrington', 'Ushuaia', 'Vila Franca Do Campo', 'Tuktoyaktuk', 'Nanakuli', 'Auki', 'Montepuez', 'Ullapool', 'Mataura', 'Mar Del Plata', 'Pereslavl-Zalesskiy', 'Alice Springs', 'Rikitea', 'Bud', 'Taolanaro', 'Puerto Ayora', 'Kruisfontein', 'Santa Catarina De Tepehuanes', 'Lagoa', 'Tasiilaq', 'Bluff', 'Saleaula', 'Kedrovoye', 'Windhoek', 'Saint-Leu', 'Pacific Grove', 'Arraial Do Cabo', 'Zhuhai', 'Perth', 'Maceio', 'Abu Samrah', 'Brae', 'Burica', 'Port Alfred',

In [4]:
# Create the Empty DataFrame for the API Calls: Columns == Temperature, Humidity, Cloudiness, Windspeed

cities_df = pd.DataFrame(cities_list)
cities_df = cities_df.rename(columns={0: "City"})
cities_df["Temperature"] = ""
cities_df["Humidity"] = ""
cities_df["Cloudiness"] = ""
cities_df["Windspeed"] = ""
cities_df

,City,Temperature,Humidity,Cloudiness,Windspeed
0,Coquimbo,,,,
1,Port Elizabeth,,,,
2,Hithadhoo,,,,
3,Veraval,,,,
4,Beloha,,,,
...,...,...,...,...,...
613,Forestville,,,,
614,Raghogarh,,,,
615,Okha,,,,
616,Taoudenni,,,,


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [5]:
# Set the base ULR Get: Temperature, Humidity, Cloudiness, Windspeed
base_weather_url = "http://api.openweathermap.org/data/2.5/weather?q="

# test city
city_test = ["Punta Arenas"]
cities_test_df = pd.DataFrame(city_test)
cities_test_df = cities_test_df.rename(columns={0: "City"})
cities_test_df["Temperature"] = ""
cities_test_df["Humidity"] = ""
cities_test_df["Cloudiness"] = ""
cities_test_df["Windspeed"] = ""
cities_test_df

,City,Temperature,Humidity,Cloudiness,Windspeed
0,Punta Arenas,,,,


In [21]:
#  for index, row in cities_pd.iterrows():

for index, row in cities_test_df.iterrows():
    time.sleep(1)
    city = row["City"]
    get_weather_url = base_weather_url + f"{city}&appid={weather_api_key}&units=imperial"
    weather_request = requests.get(get_weather_url).json()
    temp = weather_request["main"]["temp"]
    humid = weather_request["main"]["humidity"]
    cloud = weather_request["clouds"]["all"]
    wind = weather_request["wind"]["speed"]
    pprint(weather_request)
    
    cities_test_df.loc[index, "Temperature"] = temp
    cities_test_df.loc[index, "Humidity"] = humid
    cities_test_df.loc[index, "Cloudiness"] = cloud
    cities_test_df.loc[index, "Windspeed"] = wind

{'base': 'stations',
 'clouds': {'all': 20},
 'cod': 200,
 'coord': {'lat': -53.15, 'lon': -70.92},
 'dt': 1603144341,
 'id': 3874787,
 'main': {'feels_like': 19.74,
          'humidity': 72,
          'pressure': 1005,
          'temp': 42.4,
          'temp_max': 42.8,
          'temp_min': 42.01},
 'name': 'Punta Arenas',
 'sys': {'country': 'CL',
         'id': 8487,
         'sunrise': 1603099616,
         'sunset': 1603150190,
         'type': 1},
 'timezone': -10800,
 'visibility': 10000,
 'weather': [{'description': 'few clouds',
              'icon': '02d',
              'id': 801,
              'main': 'Clouds'}],
 'wind': {'deg': 260, 'speed': 34.45}}


In [22]:
cities_test_df

,City,Temperature,Humidity,Cloudiness,Windspeed
0,Punta Arenas,42.4,72,20,34.45


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression